## 1. Загрузка данных

##### 1.1 Создаём спарк сессию

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col, count, desc, lit, sum, to_date

spark = (
    SparkSession
      .builder
      .appName("HomeWork")
      .master("local[*]")
      .getOrCreate()
    )


##### 1.2 Загружаем данные из csv файла

In [2]:
logsSchema = StructType([
    StructField("ip", StringType(), True),
    StructField("timestamp", StringType(), True),
    StructField("method", StringType(), True),
    StructField("url", StringType(), True),
    StructField("response_code", IntegerType(), True),
    StructField("response_size", IntegerType(), True),
])

logs = (
    spark
      .read
      .csv("web_server_logs.csv", header=True, schema=logsSchema)
    )


##### 1.3 Выводим схему и пример загруженных данных

In [3]:
logs.printSchema()
logs.show()

root
 |-- ip: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: integer (nullable = true)
 |-- response_size: integer (nullable = true)

+---------------+--------------------+------+--------------------+-------------+-------------+
|             ip|           timestamp|method|                 url|response_code|response_size|
+---------------+--------------------+------+--------------------+-------------+-------------+
|195.242.235.226|2024-03-26T20:25:...|   PUT|       tag/blog/tags|          200|         3172|
|  11.206.37.195|2024-06-12T15:50:...|DELETE|          categories|          404|         6194|
|211.115.160.255|2024-07-23T07:25:...|  POST|        posts/search|          404|         7607|
|  78.65.249.139|2024-05-30T05:02:...|   GET|              search|          200|         9705|
|  36.249.96.122|2024-01-04T00:35:...|   PUT|main/posts/wp-con...|          500|         

## 2 Анализ данных

In [4]:
print("Top 10 active IP adresses:")
(
    logs
      .groupBy("ip")
      .agg(count("*").alias("request_count"))
      .orderBy(desc("request_count"))
      .limit(10)
).show()


print("Requests count by HTTP method:")
(
    logs
      .groupBy("method")
      .agg(count("*").alias("method_count"))
).show()


number_of_404_respose = (
    logs
      .where(col("response_code") == lit(404))
      .count()
)
print(f"Number of 404 respose codes: {number_of_404_respose}")

print()

print("Total response size by days:")
(
    logs
      .where(col("response_code") == lit(404))
      .withColumn("date", to_date("timestamp"))
      .groupBy("date")
      .agg(sum("response_size").alias("total_response_size"))
      .orderBy("date")
).show()

Top 10 active IP adresses:
+---------------+-------------+
|             ip|request_count|
+---------------+-------------+
|  90.119.242.26|          130|
| 29.249.247.181|          130|
|  177.75.189.89|          127|
|  30.14.191.240|          127|
|  152.130.23.78|          127|
|   214.41.169.9|          127|
|100.148.130.254|          127|
|  189.11.73.236|          126|
|  184.241.37.42|          125|
| 131.193.85.122|          124|
+---------------+-------------+

Requests count by HTTP method:
+------+------------+
|method|method_count|
+------+------------+
|  POST|       24967|
|DELETE|       24814|
|   PUT|       25191|
|   GET|       25028|
+------+------------+

Number of 404 respose codes: 24853

Total response size by days:
+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2024-01-01|             556153|
|2024-01-02|             484234|
|2024-01-03|             641478|
|2024-01-04|             613918|
|2024-01-05|        